# Imports

In [12]:
import pandas as pd
import os
import math
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Task 1

### Beat Drug

In [13]:
beat_drug_fp = os.path.join(os.getcwd(), "medical_genomics_2021_data", "beat_drug")
beat_drug_df = pd.read_csv(beat_drug_fp, sep='\t', lineterminator='\n')
print(beat_drug_df.shape)
beat_drug_df.head()

(79, 198)


,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
A-674563,0.302313,0.183573,10.0,2.250759,0.312102,1.112644,3.699257,1.522665,5.132768,1.83987,...,0.726144,0.454933,0.103835,0.192891,0.515219,0.531155,1.261865,0.674077,0.518016,0.259479
Afatinib (BIBW-2992),0.270839,0.095105,10.0,1.042656,0.412103,0.492191,1.126676,0.313330,4.442185,1.09213,...,1.146014,1.047075,0.556972,0.615777,0.456265,0.809377,1.279045,0.983270,2.017039,1.303088
Alisertib (MLN8237),2.292925,10.000000,10.0,10.000000,1.731348,6.149094,10.000000,10.000000,10.000000,10.00000,...,10.000000,10.000000,2.080804,8.928137,6.927002,9.425326,4.785701,10.000000,10.000000,10.000000
Axitinib (AG-013736),0.137656,10.000000,10.0,10.000000,0.124162,10.000000,10.000000,10.000000,10.000000,10.00000,...,10.000000,10.000000,10.000000,10.000000,0.418123,10.000000,10.000000,10.000000,10.000000,0.299026
AZD1480,2.429158,NaN,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,...,10.000000,6.874162,10.000000,10.000000,3.607693,7.004078,10.000000,10.000000,10.000000,10.000000


In [14]:
beat_drug_df.describe()

,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
count,69.000000,70.000000,78.000000,79.000000,79.000000,78.000000,79.000000,79.000000,79.000000,79.000000,...,79.000000,77.000000,78.000000,79.000000,77.000000,78.000000,77.000000,79.000000,79.000000,78.000000
mean,2.412734,6.335849,8.527381,7.409570,3.264417,5.883175,6.617408,6.929836,8.324497,7.984273,...,8.255971,6.404386,6.044692,6.118522,3.080462,7.342661,7.688149,7.247408,8.262772,4.240337
std,3.775972,4.299582,3.146445,3.745823,4.025590,4.275380,3.974038,4.222027,3.351549,3.474718,...,3.330924,4.001485,4.466019,4.325312,3.899469,3.839974,3.755343,3.908462,3.363649,4.264927
min,0.001400,0.006859,0.013717,0.013717,0.005953,0.013717,0.013717,0.013717,0.006859,0.189100,...,0.064853,0.013717,0.012358,0.013717,0.002706,0.097225,0.013717,0.042784,0.111219,0.010583
25%,0.118563,1.085082,10.000000,3.894494,0.248112,1.028161,2.198463,1.468053,10.000000,7.130972,...,9.681701,1.927824,0.918891,1.000000,0.220118,3.142175,5.000000,4.222760,10.000000,0.459808
50%,0.283653,10.000000,10.000000,10.000000,0.897805,6.654102,10.000000,10.000000,10.000000,10.000000,...,10.000000,9.102755,10.000000,10.000000,0.702216,10.000000,10.000000,10.000000,10.000000,1.825458
75%,2.292925,10.000000,10.000000,10.000000,7.096167,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,5.840653,10.000000,10.000000,10.000000,10.000000,10.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [15]:
beat_drug_df.isna().sum().sum()

378

#### Transformation

In [16]:
trans_beat_drug = beat_drug_df.fillna(0.0).apply(np.log10).transpose().reset_index().rename(columns={"index": "sampleID"}) # ToDo: Change fillna
trans_beat_drug = trans_beat_drug.replace(-math.inf, 0.0).replace(math.inf, 1) # ToDo: Deal with that
trans_beat_drug.head()

,sampleID,A-674563,Afatinib (BIBW-2992),Alisertib (MLN8237),Axitinib (AG-013736),AZD1480,Barasertib (AZD1152-HQPA),BEZ235,BMS-345541,Bortezomib (Velcade),...,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Trametinib (GSK1120212),Vandetanib (ZD6474),Vargetef,Vatalanib (PTK787),Vismodegib (GDC-0449),VX-745,YM-155
0,12-00066,-0.519543,-0.567290,0.360390,-0.861206,0.385456,1.0,-1.417073,0.095660,-0.744268,...,0.502356,0.869371,0.296700,-1.862741,0.216112,-1.333854,0.088790,0.0,-1.106552,-0.661768
1,12-00150,-0.736191,-1.021795,1.000000,1.000000,0.000000,1.0,1.000000,-0.102270,-0.996701,...,1.000000,1.000000,1.000000,1.000000,0.530730,-0.051009,0.409831,0.0,1.000000,-0.197125
2,12-00211,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.888241,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
3,12-00258,0.352329,0.018141,1.000000,1.000000,1.000000,1.0,0.511842,0.515246,0.310652,...,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
4,12-00294,-0.505703,-0.384994,0.238384,-0.906013,1.000000,1.0,-1.597193,-0.022079,-0.317022,...,1.000000,0.245287,-0.136855,-1.862741,0.858581,-0.771062,0.588563,1.0,-0.256898,-0.009582


### Beat RNASeq

In [17]:
beat_rnaseq_fp = os.path.join(os.getcwd(), "medical_genomics_2021_data", "beat_rnaseq")
beat_rnaseq_df = pd.read_csv(beat_rnaseq_fp, sep='\t', lineterminator='\n')
print(beat_rnaseq_df.shape)
beat_rnaseq_df.head()

(22843, 198)


,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
TSPAN6,0.070215,1.709459,0.000000,0.000000,0.000000,0.921085,0.000000,0.000000,0.000000,0.115955,...,0.052768,0.271781,0.118997,0.164221,0.648789,0.085224,0.395749,0.249059,0.142059,0.150957
DPM1,23.311415,32.137834,25.117470,29.351673,31.240770,22.940781,28.176101,34.303196,39.203413,24.089688,...,21.089525,25.547442,26.208989,32.023086,34.238344,28.607021,25.101824,38.785308,33.703438,16.575035
SCYL3,10.532266,17.778376,37.659392,15.647852,39.189992,33.734750,23.989937,17.804722,15.450059,25.510139,...,6.842223,34.244444,30.433366,26.234297,13.477109,20.823184,17.950065,7.245358,16.461058,16.212740
C1orf112,11.129094,11.253940,12.541923,8.190423,18.875313,11.110592,12.907338,6.758411,23.516117,12.465182,...,8.864988,24.520713,14.755571,13.219787,5.721136,16.874449,6.105849,4.845333,8.647826,6.944004
FGR,906.792960,49.004499,111.834945,1004.458236,267.852800,170.631056,68.186164,619.473686,11.031520,201.790997,...,451.393235,187.045931,102.961759,186.329197,583.113507,22.300408,24.508199,77.751751,45.565344,51.898882


In [18]:
beat_rnaseq_df.describe()

,12-00066,12-00150,12-00211,12-00258,12-00294,12-00372,13-00028,13-00034,13-00098,13-00118,...,16-01102,16-01138,16-01151,16-01185,16-01216,16-01219,16-01227,16-01237,16-01262,16-01270
count,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,...,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000,22843.000000
mean,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,...,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087,43.777087
std,239.036200,179.037496,199.633444,255.030349,188.159584,165.383966,181.704322,357.619163,204.698390,190.984151,...,287.677166,262.096631,261.028608,229.596117,215.486602,290.598365,498.113439,430.670074,364.906680,272.110655
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.315968,0.398874,0.437118,0.286824,0.457980,0.546894,0.295178,0.255570,0.415165,0.434832,...,0.228660,0.483167,0.475986,0.492663,0.530827,0.426122,0.339214,0.316984,0.248603,0.332105
50%,3.896938,4.558558,5.245415,3.888061,5.659323,5.871919,4.535010,3.577982,4.181302,5.536860,...,2.603211,4.922261,4.581367,5.296126,4.305597,4.460082,2.996389,3.418903,3.480439,4.106020
75%,29.560559,32.565199,37.121401,30.435231,38.601161,37.534227,36.575262,29.986899,29.891861,38.047793,...,24.035727,33.489496,32.248063,34.979063,28.119088,32.939266,20.819249,23.762511,30.862263,28.591181
max,18392.670950,8743.627777,15104.913180,25132.075360,14059.753090,8427.297469,13429.990780,44188.081700,10933.570860,12421.379490,...,17424.573900,19765.174310,16845.031580,15343.553510,13268.700230,20064.259830,42172.561690,34482.244890,42426.733510,13589.053420


In [19]:
beat_rnaseq_df.isna().sum().sum()

0

#### Transformation

In [20]:
trans_beat_rna = beat_rnaseq_df.fillna(0.0).transform(lambda x: x + 1).apply(np.log2).transpose().reset_index().rename(columns={"index": "sampleID"})
trans_beat_rna = trans_beat_rna.replace(-math.inf, 0.0).replace(math.inf, 1) # ToDo: Deal with that
trans_beat_rna.iloc[:, :100].describe()
trans_beat_rna.head()

,sampleID,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,RP11-218F10.3,RP11-686O6.2,RP11-313P22.1,RP11-548H3.1,RP11-102N12.3,RP11-196O16.1,RP4-671G15.2,RP11-731C17.2,RP4-621B10.8,RP11-114I8.4
0,12-00066,0.097901,4.603562,3.527604,3.600400,9.826219,2.520930,5.487015,5.502739,5.868784,...,0.576770,0.189578,0.275776,1.327472,0.275776,0.396124,0.144468,1.200994,0.049781,0.643137
1,12-00150,1.438005,5.050407,4.231000,3.615174,5.643986,3.211384,6.419366,5.334356,5.046681,...,0.192137,0.702038,0.192137,1.208258,1.311231,1.058426,0.000000,0.702038,0.000000,0.227674
2,12-00211,0.000000,4.706943,5.272747,3.759361,6.818070,1.640779,5.908769,6.094865,6.294083,...,1.329468,0.381416,0.182054,1.609326,0.093897,0.981932,0.093897,1.857068,0.454038,1.386245
3,12-00258,0.000000,4.923704,4.057264,3.200131,9.973637,4.972847,4.955170,5.492224,5.719514,...,0.363815,0.870742,0.173099,1.865981,0.654113,0.433561,0.089144,0.967966,0.467209,0.290525
4,12-00294,0.000000,5.010814,5.328764,4.312906,8.070673,1.760450,5.260227,5.843724,5.932174,...,1.566947,0.258552,0.297473,1.689064,0.511232,0.046439,0.297473,2.513655,0.862210,0.913222


## Join

In [21]:
beat_rna_and_drug = pd.merge(trans_beat_rna, trans_beat_drug, on="sampleID")
print(f"Num of samples in rna: {trans_beat_rna.shape[0]}\nNum of samples in drug: {trans_beat_drug.shape[0]}\nNum of samples in joined: {beat_rna_and_drug.shape[0]}")
beat_rna_and_drug.head()
print(np.isinf(beat_rna_and_drug.iloc[:, 1:]).values.sum())
beat_rna_and_drug.head()

Num of samples in rna: 198
Num of samples in drug: 198
Num of samples in joined: 198
0


,sampleID,TSPAN6,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,TG100-115,Tofacitinib (CP-690550),Tozasertib (VX-680),Trametinib (GSK1120212),Vandetanib (ZD6474),Vargetef,Vatalanib (PTK787),Vismodegib (GDC-0449),VX-745,YM-155
0,12-00066,0.097901,4.603562,3.527604,3.600400,9.826219,2.520930,5.487015,5.502739,5.868784,...,0.502356,0.869371,0.296700,-1.862741,0.216112,-1.333854,0.088790,0.0,-1.106552,-0.661768
1,12-00150,1.438005,5.050407,4.231000,3.615174,5.643986,3.211384,6.419366,5.334356,5.046681,...,1.000000,1.000000,1.000000,1.000000,0.530730,-0.051009,0.409831,0.0,1.000000,-0.197125
2,12-00211,0.000000,4.706943,5.272747,3.759361,6.818070,1.640779,5.908769,6.094865,6.294083,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
3,12-00258,0.000000,4.923704,4.057264,3.200131,9.973637,4.972847,4.955170,5.492224,5.719514,...,1.000000,-1.862741,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,0.000000
4,12-00294,0.000000,5.010814,5.328764,4.312906,8.070673,1.760450,5.260227,5.843724,5.932174,...,1.000000,0.245287,-0.136855,-1.862741,0.858581,-0.771062,0.588563,1.0,-0.256898,-0.009582


## Linear Regression

In [22]:
t_1_LR = LinearRegression()
num_of_features = beat_rnaseq_df.shape[0]
print(f"Num of features (genes): {num_of_features}")
X = beat_rna_and_drug.iloc[:, 1:num_of_features + 1]
y = beat_rna_and_drug.iloc[:, num_of_features + 2]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
t_1_LR.fit(x_train, y_train)
y_pred = t_1_LR.predict(x_test)
score = r2_score(y_test, y_pred)
print(score)

Num of features (genes): 22843
-0.22014216665460618
